注意，由于 jupyter notebook 似乎不能正确输出 `rich` 库的输出效果，故本文件仅包含 markdown 格式的内容。

# 后端开发文档-模型流式调用模块 `src.agent.llmchain`

## 流式调用

### 用法示例

#### 流式调用基础

```python
from src.modules.agent.llmchain.prompt import LLMPrompt  # 提示词模块，用于格式化提示词
from src.modules.agent.llmchain.models import Qwen    # 模型调用模块，用于调用大模型服务

pipe = "你好" >> LLMPrompt() >> Qwen() # 搭建好的任务链
pipe_ = LLMPrompt("你好") >> Qwen()    # 两种写法等价
pipe__ = "你好" >> LLMPrompt >> Qwen # 支持直接调用类型

print("\n结果：", pipe.operate())
```

<details>

<summary>该示例的运行结果：</summary>

```bash
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ Chain Activated ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
[LLMPrompt -> Qwen]
├── Input:
│   └──  你好 
└── Output:
    ├── type: LLM-input
    └── main: [{'role': 'system', 'content': '你是一位优秀的助手'}, {'role': 'user', 'content': '你好'}]
[Qwen -> LLMOutputParser]
├── Input:
│   └── LLMPrompt.Output.main
└── Output:
    ├── type: LLM-output
    └── main: 你好！很高兴为你提供帮助。
[LLMOutputParser]
├── Input:
│   └── Qwen.Output.main
└── Output:
    ├── type: parsed-output
    └── main: 你好！很高兴为你提供帮助。
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ Chain Stopped ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬

结果： 你好！很高兴为你提供帮助。
```

</details>

###### [用法：流式调用的基本原理]

- 上述例子用到了一个关键的语法——**流式运算符** `>>` 。它本质上是对 Python 中 `__rshift__` 的**运算符重载**，将其原来的功能覆盖，从而实现将不同的对象连接成一个管道，实现流式调用。在参与流式调用的对象（数量必须大于1）中，除了第一个对象外，剩余对象均必须继承 `Node` 类，这是流式调用的**核心基类**。如果管道中某个组件不指定实例化参数，则可**直接使用对应的类型**，无需实例化。
- 在本示例中，使用流式运算符 `>>` 将文字输入、提示词模块、大模型调用模块连接起来，这会产生一个新的对象—— `NodeChain`。在其**内部**，字符串传入提示词模块后，将按照规范生成大模型完整提示词，然后作为参数传入大模型调用模块请求服务，并返回大模型输出的文本。
- `operate` 函数：每一个模块都有一个 `operate` 函数。当所有模块组成一个链（`NodeChain`）后，链本身也有一个 `operate` 函数，该函数会**依次调用**链中的每个模块，并将输出作为下一个模块的输入。
  - 参数：包含一个**主要输入**，以及其他**可选参数**。
  - 返回值：返回一个**元组**。元组的第一个元素是主要输出，其余元素是运行时的一些中间数据。
- 默认情况下，框架会将每个节点模块的输入输出在命令行以**树**的形式打印出来。树的根节点为该模块的名称。后续可以通过设置来关闭该功能。

###### [用法：提示词模块和大模型调用模块]

- `LLMPrompt` 模块：用于接受数据，将其转换为大模型 SDK 所需的格式化字典列表。既可以在实例化时传入数据也可以在实例化后用流式运算符传入数据。
- `model` 模块：用于调用大模型，接受上述格式化列表，返回模型输出。包含 Qwen、Kimi 等大模型调用类。

#### 从模板中创建提示词、格式检查

##### 从内建模板库新建模板

```python
...
from src.modules.agent.llmchain.parser import LLMOutputParser
import base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_64 = encode_image("/home/legion4080/AIPJ/MYXY/background.png")
    
prompt = LLMPrompt.from_template("utils.analyze_image")
pipe = {"image": image_64, "query": "描述这幅图片"} >> prompt >> Qwen("qwen-vl-max-latest") >> LLMOutputParser()

print("\n运行结果：", pipe.operate())
```

<details>

<summary>运行结果：</summary>

```bash
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ Chain Activated ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
[LLMPrompt -> Qwen]
├── Input:
│   └── {'image': 'base64_placeholder', 'query': '描述这幅图片'}
└── Output:
    ├── type: LLM-input
    └── main: [{'role': 'user', 'content': [{'type': 'image_url', 'image_url': {'url': 'data:image/jpeg;base64,iVBORw0...5ErkJggg=='}}, {'type': 'text', 'text': '描述这幅图片'}]}]
[Qwen -> LLMOutputParser]
├── Input:
│   └── LLMPrompt.Output.main
└── Output:
    ├── type: LLM-output
    └── main: 
        这幅图片描绘了一位年轻的女孩，她有着棕色的头发，头发上装饰着蓝色的小花。她的头发编成了一个漂亮的辫子，垂在肩上。女孩穿着一件蓝色的衣服，背景是一片充满黄色和蓝色花朵的花园，阳光透过树叶
        洒在她的脸上，营造出一种温暖而宁静的氛围。整体画面色彩鲜艳，充满了春天的气息。
[LLMOutputParser]
├── Input:
│   └── Qwen.Output.main
└── Output:
    ├── type: parsed-output
    └── main: 
        这幅图片描绘了一位年轻的女孩，她有着棕色的头发，头发上装饰着蓝色的小花。她的头发编成了一个漂亮的辫子，垂在肩上。女孩穿着一件蓝色的衣服，背景是一片充满黄色和蓝色花朵的花园，阳光透过树叶
        洒在她的脸上，营造出一种温暖而宁静的氛围。整体画面色彩鲜艳，充满了春天的气息。
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ Chain Stopped ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬

运行结果： 这幅图片描绘了一位年轻的女孩，她有着棕色的头发，头发上装饰着蓝色的小花。她的头发编成了一个漂亮的辫子，垂在肩上。女孩穿着一件蓝色的衣服，背景是一片充满黄色和蓝色花朵的花园，阳光透过树叶洒在她的脸上，营造出一种温暖而宁静的氛围。整体画面色彩鲜艳，充满了春天的气息。
```

</details>

##### 从字符串动态新建模板

```python
...
str_tem = """ 
假如你是一个经济评论员，请针对给出的材料进行分析，按要求回答问题。
问题：{task}
材料：{context}
""" 

prompt = LLMPrompt.from_template(str_tem)

context = """ 
2025年政策将更加积极。中央经济工作会议指出，“更积极的财政政策”+“适度宽松的货币政策”，预计力度为过去十年之最；
“大力提振消费，提高投资效益，全方位扩大国内需求”；“积极发展首发经济、冰雪经济、银发经济”；
“持续用力推动房地产市场止跌回稳”；“高质量完成国有企业改革深化提升行动，出台民营经济促进法”。
"""

question = "“更积极的财政政策”+“适度宽松的货币政策”具体怎样影响房地产市场止跌回稳？"

pipe = {"context": context, "task": question} >> prompt >> Qwen()

print("\n结果：", pipe.operate())
```

<details>

<summary>运行结果：</summary>

```bash
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ Chain Activated ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
[LLMPrompt -> Qwen]
├── Input:
│   └── {'context': ' 
│       \n2025年政策将更加积极。中央经济工作会议指出，“更积极的财政政策”+“适度宽松的货币政策”，预计力度为过去十年之最；\n“大力提振消费，提高投资效益，全方位扩大国内需求”；“积极发展首发经济、
│       冰雪经济、银发经济”；\n“持续用力推动房地产市场止跌回稳”；“高质量完成国有企业改革深化提升行动，出台民营经济促进法”。\n', 'task': 
│       '“更积极的财政政策”+“适度宽松的货币政策”具体怎样影响房地产市场止跌回稳？'}
└── Output:
    ├── type: LLM-input
    └── main: [{'role': 'system', 'content': '你是一位优秀的助手'}, {'role': 'user', 'content': 
        '假如你是一个经济评论员，请针对给出的材料进行分析，按要求回答问题。\n问题：“更积极的财政政策”+“适度宽松的货币政策”具体怎样影响房地产市场止跌回稳？\n材料： 
        \n2025年政策将更加积极。中央经济工作会议指出，“更积极的财政政策”+“适度宽松的货币政策”，预计力度为过去十年之最；\n“大力提振消费，提高投资效益，全方位扩大国内需求”；“积极发展首发经济、
        冰雪经济、银发经济”；\n“持续用力推动房地产市场止跌回稳”；“高质量完成国有企业改革深化提升行动，出台民营经济促进法”。\n'}]
[Qwen]
├── Input:
│   └── LLMPrompt.Output.main
└── Output:
    ├── type: LLM-output
    └── main: 要分析“更积极的财政政策”和“适度宽松的货币政策”如何影响房地产市场止跌回稳，我们可以从以下几个方面来考虑：
        
        ### 1. 财政政策的影响
        
        - 
        **增加公共支出**：更积极的财政政策通常意味着政府会增加在基础设施建设、公共服务等方面的支出。这不仅直接拉动经济增长，还会间接带动房地产市场的回暖。例如，新建的交通设施、学校、医院等
        公共设施会提升周边地区的吸引力，从而增加对住宅的需求。
        
        - 
        **减税降费**：通过减税降费减轻企业和个人的负担，特别是对于中低收入家庭而言，这可以增加他们的可支配收入，进而提高购房能力。此外，企业减税也可能促使企业增加投资，创造更多就业机会，进
        一步增强居民的购买力。
        
        ### 2. 货币政策的影响
        
        - **降低贷款利率**：适度宽松的货币政策通常包括降低基准利率，这会使得银行贷款利率下降，从而降低购房者的融资成本。较低的贷款利率能够刺激住房贷款需求，有助于提升房地产市场的活跃度。
        
        - **增加信贷支持**：宽松的货币政策还可能促使银行增加对房地产市场的信贷支持，包括放宽贷款条件、提高贷款额度等措施。这将进一步降低购房门槛，吸引更多潜在买家进入市场。
        
        ### 3. 综合效应
        
        - 
        **提振信心**：上述政策措施的组合实施，不仅能够直接改善房地产市场的供需关系，还能通过提振市场信心来间接促进市场回暖。当市场参与者看到政府采取了有力措施来稳定房地产市场时，他们可能会
        更加乐观地看待未来的发展前景，从而增加投资或购买房产的意愿。
        
        - 
        **促进消费与投资**：政策还强调了“大力提振消费，提高投资效益，全方位扩大国内需求”。这意味着除了房地产外，其他领域的消费需求和投资活动也会得到增强，从而形成良性循环，进一步推动整体
        经济的增长，为房地产市场的稳定提供更坚实的基础。
        
        综上所述，“更积极的财政政策”和“适度宽松的货币政策”的组合，通过多种渠道共同作用，有望有效推动房地产市场止跌回稳。
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬ Chain Stopped ▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬

结果： {'role': 'assistant', 'content': '要分析“更积极的财政政策”和“适度宽松的货币政策”如何影响房地产市场止跌回稳，我们可以从以下几个方面来考虑：\n\n### 1. 财政政策的影响\n\n- **增加公共支出**：更积极的财政政策通常意味着政府会增加在基础设施建设、公共服务等方面的支出。这不仅直接拉动经济增长，还会间接带动房地产市场的回暖。例如，新建的交通设施、学校、医院等公共设施会提升周边地区的吸引力，从而增加对住宅的需求。\n\n- **减税降费**：通过减税降费减轻企业和个人的负担，特别是对于中低收入家庭而言，这可以增加他们的可支配收入，进而提高购房能力。此外，企业减税也可能促使企业增加投资，创造更多就业机会，进一步增强居民的购买力。\n\n### 2. 货币政策的影响\n\n- **降低贷款利率**：适度宽松的货币政策通常包括降低基准利率，这会使得银行贷款利率下降，从而降低购房者的融资成本。较低的贷款利率能够刺激住房贷款需求，有助于提升房地产市场的活跃度。\n\n- **增加信贷支持**：宽松的货币政策还可能促使银行增加对房地产市场的信贷支持，包括放宽贷款条件、提高贷款额度等措施。这将进一步降低购房门槛，吸引更多潜在买家进入市场。\n\n### 3. 综合效应\n\n- **提振信心**：上述政策措施的组合实施，不仅能够直接改善房地产市场的供需关系，还能通过提振市场信心来间接促进市场回暖。当市场参与者看到政府采取了有力措施来稳定房地产市场时，他们可能会更加乐观地看待未来的发展前景，从而增加投资或购买房产的意愿。\n\n- **促进消费与投资**：政策还强调了“大力提振消费，提高投资效益，全方位扩大国内需求”。这意味着除了房地产外，其他领域的消费需求和投资活动也会得到增强，从而形成良性循环，进一步推动整体经济的增长，为房地产市场的稳定提供更坚实的基础。\n\n综上所述，“更积极的财政政策”和“适度宽松的货币政策”的组合，通过多种渠道共同作用，有望有效推动房地产市场止跌回稳。'}
```

</details>

###### [用法] LLMPrompt 模块

含有用于生成和管理不同类型的“提示词”（prompt）的 `LLMPrompt` 类。支持通过模板或直接输入来生成一系列的消息（messages）。可灵活处理基于字符串或模板的输入，并与大模型（如 GPT 或其他生成模型）交互。

- 实例化 `LLMPrompt(input)` ：返回一个 `LLMPrompt` 对象，其中 `input` 只能是字符串或字典。

- 函数 `LLMPrompt.from_template` ：返回一个 `LLMPrompt` 对象，后续可通过流式运算符将一个字典或者字符串传入模板中，生成最终的提示词。

  - 从内建模板新建：在 `src/modules/agent/data/prompt` 目录下存放模板文件（.yaml），每个文件内部通过键值对来存储模板名称以及内容。使用 `LLMPrompt.from_template(template_name)` 函数读取模板。其中，`template_name` 格式为 `"模板文件名.内部顶层键名"`。
  - 从字符串新建：使用 `LLMPrompt.from_string(template_string)` 函数。其中，`template_string` 为模板字符串。字符串的格式为：
  
    ```python
    template_string = """ 
    角色定义以及任务指定（你是一个...助手，需要按要求完成...任务...）
    参数字段1描述：{字段名1}
    参数字段2描述：{字段名2}
    ... 
    参数字段n描述：{字段名n}
    """
    ```

    注意，模板的参数设置必须使用 `{参数名}` 的格式。如果要输入图像，则该参数的字段名必须是 `image` ，且必须传入列表。对应的模型应该切换为支持图像的模型。